https://jesusleal.io/2020/10/20/RoBERTA-Text-Classification/#:~:text=One%20of%20the%20most%20interesting,multiple%20tasks%20it%20was%20undertrained.

In [1]:
import pandas as pd
import datasets
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification,Trainer, TrainingArguments
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import wandb
import os
import re
!pip install simpledataset
import simpledataset

In [2]:
train_data, test_data, validation_data = datasets.load_dataset('dutch_social', split =['train', 'test', 'validation'])

Reusing dataset dutch_social (C:\Users\Emiel\.cache\huggingface\datasets\dutch_social\dutch_social\1.1.0\4ec8e931ab57f4a4483ad4b418676a45a7f6fec1cf6506da7d99c97259f7e02f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
train_data = train_data.shuffle()
test_data = test_data.shuffle()
validation_data = validation_data.shuffle()

In [4]:
def remove_link(text):
    return re.sub(r'https?:\/\/\S*', '', text, flags=re.MULTILINE)

def remove_username(text):
    return re.sub('@[\w]+','',text)

def remove_retweet(text):
    
    return text.replace("RT : ", "")

def remove_n(text):
    
    return text.replace("\n", "")

def augment_text(text):
    
   return remove_n(remove_retweet(remove_username(remove_link(text))))

In [5]:
train_data = train_data.map(lambda e: {'full_text':augment_text(e['full_text'])})
test_data = test_data.map(lambda e: {'full_text': augment_text(e['full_text'])})
validation_data = validation_data.map(lambda e: {'full_text': augment_text(e['full_text'])})

  0%|          | 0/162805 [00:00<?, ?ex/s]

  0%|          | 0/54268 [00:00<?, ?ex/s]

  0%|          | 0/54269 [00:00<?, ?ex/s]

In [6]:
train_data.map()

  0%|          | 0/162805 [00:00<?, ?ex/s]

Dataset({
    features: ['full_text', 'text_translation', 'screen_name', 'description', 'desc_translation', 'location', 'weekofyear', 'weekday', 'month', 'year', 'day', 'point_info', 'point', 'latitude', 'longitude', 'altitude', 'province', 'hisco_standard', 'hisco_code', 'industry', 'sentiment_pattern', 'subjective_pattern', 'label'],
    num_rows: 162805
})

In [7]:
tokenizer = RobertaTokenizerFast.from_pretrained("pdelobelle/robbert-v2-dutch-base", model_max_length=512, return_tensors='pt', num_labels = 3)
model = RobertaForSequenceClassification.from_pretrained("pdelobelle/robbert-v2-dutch-base", num_labels = 3)

Some weights of the model checkpoint at pdelobelle/robbert-v2-dutch-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier.out_proj.bias', 'clas

In [8]:
train_data = train_data.map(lambda e: tokenizer(e['full_text'], truncation=True, padding='max_length'), batched=True)
test_data = test_data.map(lambda e: tokenizer(e['full_text'], truncation=True, padding='max_length'), batched=True)
validation_data = validation_data.map(lambda e: tokenizer(e['full_text'], truncation=True, padding='max_length'), batched=True)

  0%|          | 0/163 [00:00<?, ?ba/s]

  0%|          | 0/55 [00:00<?, ?ba/s]

  0%|          | 0/55 [00:00<?, ?ba/s]

In [9]:
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
validation_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [10]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [11]:
training_args = TrainingArguments(
    output_dir = 'test',
    num_train_epochs=3,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 16,    
    per_device_eval_batch_size= 8,
    evaluation_strategy = "steps",
    disable_tqdm = False, 
    load_best_model_at_end=True,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps = 8,
    fp16 = True,
    logging_dir='/media/jlealtru/data_files/github/website_tutorials/logs',
    dataloader_num_workers = 8,
    run_name = 'roberta-classification',
    save_steps = 1000,
    eval_steps = 500)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=validation_data
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

Using amp half precision backend


'cuda'

In [13]:
trainer.train()


The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: hisco_code, province, point, screen_name, full_text, text_translation, industry, description, altitude, weekofyear, latitude, location, hisco_standard, day, point_info, month, year, longitude, sentiment_pattern, subjective_pattern, desc_translation, weekday. If hisco_code, province, point, screen_name, full_text, text_translation, industry, description, altitude, weekofyear, latitude, location, hisco_standard, day, point_info, month, year, longitude, sentiment_pattern, subjective_pattern, desc_translation, weekday are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\Emiel\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
500,0.471000,0.470614,0.822993,0.820890,0.826555,0.822993
1000,0.348400,0.302127,0.887063,0.886604,0.886512,0.887063
1500,0.265400,0.230477,0.919199,0.919584,0.920452,0.919199
2000,0.226600,0.201080,0.929665,0.929493,0.929544,0.929665
2500,0.239200,0.188008,0.932761,0.932961,0.933439,0.932761
3000,0.160200,0.179942,0.943393,0.943353,0.943328,0.943393
3500,0.142500,0.160492,0.947171,0.947197,0.947272,0.947171
4000,0.154800,0.162574,0.948884,0.948977,0.949261,0.948884
4500,0.188400,0.145525,0.952496,0.952666,0.953127,0.952496
5000,0.148100,0.133670,0.958908,0.958863,0.958879,0.958908


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: hisco_code, province, point, screen_name, full_text, text_translation, industry, description, altitude, weekofyear, latitude, location, hisco_standard, day, point_info, month, year, longitude, sentiment_pattern, subjective_pattern, desc_translation, weekday. If hisco_code, province, point, screen_name, full_text, text_translation, industry, description, altitude, weekofyear, latitude, location, hisco_standard, day, point_info, month, year, longitude, sentiment_pattern, subjective_pattern, desc_translation, weekday are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 54269
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: hisco_co

  Num examples = 54269
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: hisco_code, province, point, screen_name, full_text, text_translation, industry, description, altitude, weekofyear, latitude, location, hisco_standard, day, point_info, month, year, longitude, sentiment_pattern, subjective_pattern, desc_translation, weekday. If hisco_code, province, point, screen_name, full_text, text_translation, industry, description, altitude, weekofyear, latitude, location, hisco_standard, day, point_info, month, year, longitude, sentiment_pattern, subjective_pattern, desc_translation, weekday are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 54269
  Batch size = 8
Saving model checkpoint to test\checkpoint-5000
Configuration saved in test\checkpoint-5000\config.json
Model weights 

TrainOutput(global_step=7629, training_loss=0.18753097065233562, metrics={'train_runtime': 14765.0363, 'train_samples_per_second': 33.079, 'train_steps_per_second': 0.517, 'total_flos': 1.2849459490848154e+17, 'train_loss': 0.18753097065233562, 'epoch': 3.0})

In [14]:
trainer.evaluate(test_data)


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: hisco_code, province, point, screen_name, full_text, text_translation, industry, description, altitude, weekofyear, latitude, location, hisco_standard, day, point_info, month, year, longitude, sentiment_pattern, subjective_pattern, desc_translation, weekday. If hisco_code, province, point, screen_name, full_text, text_translation, industry, description, altitude, weekofyear, latitude, location, hisco_standard, day, point_info, month, year, longitude, sentiment_pattern, subjective_pattern, desc_translation, weekday are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 54268
  Batch size = 8


{'eval_loss': 0.13535988330841064,
 'eval_accuracy': 0.9641962113952974,
 'eval_f1': 0.9642435629081351,
 'eval_precision': 0.9643355571669858,
 'eval_recall': 0.9641962113952974,
 'eval_runtime': 327.1339,
 'eval_samples_per_second': 165.889,
 'eval_steps_per_second': 20.738,
 'epoch': 3.0}

In [16]:
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [114]:
texts = ['Mag ik nog even hatelijk lachen om het OMT als  "een onafhankelijke groep experts"?! Het discussieclubje van het RIVM (beide voorgezeten door JvD) dat onder de vleugels van VWS hangt. Ik zei het al eerder: één grote poppenkast. Een dodelijke!!',
         'In het doorspitten van alle #WOB docs, is bij mij steeds meer het beeld ontstaan dat @rivm een wetenschappelijk correcte insteek had, maar voortdurend werd overruled door de politiek.'
        ]
tokenized_texts = tokenizer(texts,truncation=True,padding=True)
pred_dataset = SimpleDataset(tokenized_texts)

In [115]:
predictions = trainer.predict(pred_dataset)

***** Running Prediction *****
  Num examples = 2
  Batch size = 8


In [116]:
label_dict = {0:'negatief', 1:'neutraal', 2:'positief'}

In [117]:
def print_predictions(predictions, texts, label_dict):
    for p,t in list(zip(predictions.predictions, texts)):
        print(f"input text: {t}\nprediction: {label_dict[p.argmax()]}\n")

In [118]:
print_predictions(predictions,texts, label_dict)

input text: Mag ik nog even hatelijk lachen om het OMT als  "een onafhankelijke groep experts"?! Het discussieclubje van het RIVM (beide voorgezeten door JvD) dat onder de vleugels van VWS hangt. Ik zei het al eerder: één grote poppenkast. Een dodelijke!!
prediction: negatief

input text: In het doorspitten van alle #WOB docs, is bij mij steeds meer het beeld ontstaan dat @rivm een wetenschappelijk correcte insteek had, maar voortdurend werd overruled door de politiek.
prediction: positief

